In [ ]:
# make sure that mongo server is running 

In [ ]:
from pymongo import MongoClient
import os
import urllib.request
from multiprocessing.pool import ThreadPool as Pool
import utils.const as const

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

In [ ]:
db=const.grpId
collection=const.PostCollection
directory='attachment'

failedFile='failed.txt'
failed=[]
pool_size=200  ## directly proportional to network bandwidth

In [ ]:
if not os.path.isdir(directory):
    os.mkdir(directory)
else:
  os.system('sudo rm -r '+directory)
  os.mkdir(directory)

In [ ]:
myCollect=client[db][collection]

In [ ]:
def download(url,fileName):
  try:
    print(fileName)
    urllib.request.urlretrieve(url, os.path.join(directory,fileName))
  except Exception as e:
    print(e)
    failed.append([fileName,url])

In [ ]:
pool = Pool(pool_size)

In [ ]:
failed=[]
for post in myCollect.find({}):
  if '_id' and 'attachments' in post:

    if 'images' in post['attachments']:
      for i in range(len(post['attachments']['images'])):
        fileName=post['_id']+'_I'+str(i)
        
        pool.apply_async(download,(post['attachments']['images'][i],fileName))

    if 'videos' in post['attachments']:
      for i in range(len(post['attachments']['videos'])):
        fileName=post['_id']+'_V'+str(i)
        
        pool.apply_async(download,(post['attachments']['videos'][i],fileName))
  
  if '_id' and 'comments' in post:
    for comment in post['comments']:
      if '_id' and 'attachment' in comment:

        if 'images' in comment['attachment']:
          for i in range(len(comment['attachment']['images'])):
            fileName=post['_id']+'_'+comment['_id']+'_I'+str(i)
            
            pool.apply_async(download,(comment['attachment']['images'][i],fileName))

        if 'videos' in comment['attachment']:
          for i in range(len(comment['attachment']['videos'])):
            fileName=post['_id']+'_'+comment['_id']+'_V'+str(i) 
            
            pool.apply_async(download,(comment['attachment']['videos'][i],fileName))


In [ ]:
pool.close()
pool.join()

In [ ]:
print(len(failed))

In [ ]:
with open('failed.txt','w') as fp:
  for arr in failed:
    fp.write(arr[0]+' '+arr[1]+' \n')
failed=[]
    
while 1:
  dictionary={}
  with open('failed.txt','r') as fp:
    for line in fp.readlines():
      arr=line.split(' ')
      dictionary[arr[0]]=arr[1]

  lastSize=len(dictionary)
  pool = Pool(pool_size)
    
  for k,v in dictionary.items():
    pool.apply_async(download,(v,k))
  
  pool.close()
  pool.join()

  with open('failed.txt','w') as fp:
    for arr in failed:
      fp.write(arr[0]+' '+arr[1]+' \n')
  
  if len(failed)==lastSize :
    break
  failed=[]



In [ ]:
print(len(failed)) ## if its non zero that means some links are expired

In [ ]:
if len(failed):
    print('run second notebook')
else:
    print('done... skip to 3rd')